# Overview
In this notebook, I used negex to detect negated entities. Negex was developed in early 2000', but it continues to be a popular package for negation detection. 

Here, we combine negex with stanza, which is an NLP package. Together, they will identify medical entities, and tell us whether an entity is negated.

# Installation

In [2]:
# import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m pip install negspacy
# !{sys.executable} -m pip install spacy_stanza #This package wraps the Stanza (formerly StanfordNLP) library, so you can use Stanford's models in a spaCy pipeline.

In [1]:
import spacy # to build a nlp pipeline
import stanza # for named entity recognition
# this package wraps Stanza around Spacy, so that we can use Stanza in a spaCy pipeline.
import spacy_stanza
from negspacy.negation import Negex
from negspacy.termsets import termset # to customize negation terms
import pandas as pd



# Set up NLP pipeline

In [2]:
# download and initialize a mimic pipeline with an i2b2 NER model
# stanza.download('en', package='mimic', processors={'ner': 'i2b2'})
nlp = spacy_stanza.load_pipeline('en', package='mimic', processors={'ner': 'i2b2'})

2021-12-17 15:04:45 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | mimic   |
| pos       | mimic   |
| lemma     | mimic   |
| depparse  | mimic   |
| ner       | i2b2    |

2021-12-17 15:04:45 INFO: Use device: cpu
2021-12-17 15:04:45 INFO: Loading: tokenize
2021-12-17 15:04:45 INFO: Loading: pos
2021-12-17 15:04:45 INFO: Loading: lemma
2021-12-17 15:04:46 INFO: Loading: depparse
2021-12-17 15:04:46 INFO: Loading: ner
2021-12-17 15:04:46 INFO: Done loading processors!


# Add customized terms to the default list of terms

In [3]:
ts = termset("en_clinical")
# customize the term list by adding more negation terms
ts.add_patterns({
            'preceding_negations': ['abstain from','other than','except for','except','with the exception of',
                                    'excluding','lack of','contraindication','contraindicated','interfere with',
                                   'prohibit','prohibits'],
            'following_negations':['negative','is allowed','impossible','exclusionary']
        })

# Let negex know what entities we are extracting

In [4]:
nlp.add_pipe("negex", config={"ent_types":["PROBLEM","TEST",'TREATMENT']})


# Examples
"True" means an entity should be negated

In [5]:
doc = nlp('Patient had a headache, but no fever')

for e in doc.ents:
	print(e.text, e._.negex)

a headache False
fever True


In [26]:
doc = nlp('No history of diabetes')

for e in doc.ents:
	print(e.text, e._.negex)

diabetes True


In [8]:
doc = nlp('Patients should abstain from painkillers like NSAIDs and allergy medications for 24 hours')

for e in doc.ents:
	print(e.text, e._.negex)

painkillers True
NSAIDs True
allergy medications True


In [10]:
doc = nlp('Women with pregnancy should not take hormonal birth control')

for e in doc.ents:
	print(e.text, e._.negex)

pregnancy False
hormonal birth control True


# References
Negation
* https://github.com/jenojp/negspacy
* https://medium.com/@MansiKukreja/clinical-text-negation-handling-using-negspacy-and-scispacy-233ce69ab2ac
* https://towardsdatascience.com/clinical-notes-the-negative-story-e1140dd275c7
* https://www.youtube.com/watch?v=IiD3YZkkCmE&t=2210s
(see 36:41 of the video)

Negex: how to add and delete custom negation terms
* https://pypi.org/project/negspacy/
